<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!gdown --folder https://drive.google.com/drive/folders/1aWSFb-ZR8TTIDdRlDBBCh-YvvCxmt6Bc

Retrieving folder contents
Processing file 1ppSSIuo0o740Z-Wfz6m47xQtmHd08znO bitsandbytes-0.43.0.dev0-cp311-cp311-win_amd64.whl
Processing file 1I6I0pdL4QNcozYaWng7i1GnmPmqwWJZC deepspeed-0.13.1+unknown-py3-none-any.whl
Processing file 1xU2jfkljiVuqqQjCkYPVa1KUFR8QF_SB llvm-5e5a22ca-windows-x64.tar.gz
Processing file 1Xt_XGgc_wgl1f079oSjDNfCBhd7iXnzC triton-2.1.0-cp311-cp311-win_amd64.whl
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1ppSSIuo0o740Z-Wfz6m47xQtmHd08znO
To: /content/311Wheels/bitsandbytes-0.43.0.dev0-cp311-cp311-win_amd64.whl
100% 24.1M/24.1M [00:00<00:00, 60.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1I6I0pdL4QNcozYaWng7i1GnmPmqwWJZC
To: /content/311Wheels/deepspeed-0.13.1+unknown-py3-none-any.whl
100% 961k/961k [00:00<00:00, 129MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1xU2jfkljiVuqqQjCkYPVa1KUFR8QF_SB
From (redirected

In [7]:
from google.colab import files
files.download('/content/311Wheels/triton-2.1.0-cp311-cp311-win_amd64.whl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import random
from datasets import Dataset, DatasetDict
from transformers import TrainingArguments
from trl import SFTTrainer
import os
from transformers import set_seed as transformers_set_seed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = r"\llama3\Llama-3-8b-Instruct"

HAS_BFLOAT16 = torch.cuda.is_bf16_supported()
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



source_train_file_ko_en = "ko_en.txt"
target_train_file_ko_en = "en_ko.txt"
source_train_file_vi_en = "vi_en.txt"
target_train_file_vi_en = "en_vi.txt"
source_train_file_vi_ko = "vi_ko.txt"
target_train_file_vi_ko = "ko_vi.txt"

with open(source_train_file_ko_en, encoding="utf-8") as source0, open(target_train_file_ko_en, encoding="utf-8") as target0:
  source_sentences0 = [sent0.strip() for sent0 in source0.readlines()]
  target_sentences0 = [sent0.strip() for sent0 in target0.readlines()]

with open(source_train_file_vi_en, encoding="utf-8") as source1, open(target_train_file_vi_en, encoding="utf-8") as target1:
  source_sentences1 = [sent1.strip() for sent1 in source1.readlines()]
  target_sentences1 = [sent1.strip() for sent1 in target1.readlines()]

with open(source_train_file_vi_ko, encoding="utf-8") as source2, open(target_train_file_vi_ko, encoding="utf-8") as target2:
  source_sentences2 = [sent2.strip() for sent2 in source2.readlines()]
  target_sentences2 = [sent2.strip() for sent2 in target2.readlines()]

def create_prompt():
    prompts = []
    #for i in range(0, len(source_sentences)):
    for i in range(0, 100):
      prompt_koen = "[INST]"+" Translate the Korean input text into English: " + source_sentences0[i] + " [/INST] " + target_sentences0[i] + " </s>"
      prompt_vien = "[INST]"+" Translate the Vietnamese input text into English: " + source_sentences1[i] + " [/INST] " + target_sentences1[i] + " </s>"
      prompt_enko = "[INST]"+" Translate the English input text into Korean: " + target_sentences0[i] + " [/INST] " + source_sentences0[i] + " </s>"
      prompt_envi = "[INST]"+" Translate the English input text into Vietnamese: " + target_sentences1[i] + " [/INST] " + source_sentences1[i] + " </s>"

      print(prompt_vien)
      prompts.append(prompt_koen)
      prompts.append(prompt_vien)
      prompts.append(prompt_enko)
      prompts.append(prompt_envi)
    for i in range(0,len(source_train_file_vi_ko)):
      prompt_kovi = "[INST]"+" Translate the Korean input text into Vietnamese: " + target_sentences2[i] + " [/INST] " + source_sentences2[i] + " </s>"
      prompt_viko = "[INST]"+" Translate the Vietnamese input text into Korean: " + source_sentences2[i] + " [/INST] " + target_sentences2[i] + " </s>"
      prompts.append(prompt_kovi)
      prompts.append(prompt_viko)
    return prompts

prompts = create_prompt()
random.shuffle(prompts)


dataset = DatasetDict({
    "train": Dataset.from_dict({"text": prompts[:len(prompts)]}),
    "validation": Dataset.from_dict({"text": prompts[int(0.9*len(prompts)):]})
})


random_state = 3407
transformers_set_seed(random_state)

output_directory = "llama3_20240711"

training_args = TrainingArguments(
                                  output_dir = output_directory,
                                  num_train_epochs=2,
                                  #max_steps = 594, # comment out this line if you want to train in epochs
                                  per_device_train_batch_size = 2,
                                  gradient_accumulation_steps = 1,
                                  optim="paged_adamw_8bit",
                                  fp16 = not HAS_BFLOAT16,
                                  bf16 = HAS_BFLOAT16,
                                  save_steps=1000,
                                  logging_steps=50,
                                  # save_strategy="epoch",
                                  evaluation_strategy=None,
                                  learning_rate=2e-4,
                                  weight_decay=0.001,
                                  max_grad_norm=0.3,
                                  max_steps=-1,
                                  warmup_ratio=0.03,
                                  group_by_length=True,
                                  lr_scheduler_type="linear",
                                  report_to=None,
                                  seed=random_state
                                )

max_seq_length = 1024  # increase if needed

trainer = SFTTrainer(
                    model=model,
                    # peft_config=lora_config,
                    max_seq_length=max_seq_length,
                    tokenizer=tokenizer,
                    packing=False,
                    dataset_text_field="text",
                    args=training_args,
                    train_dataset=dataset["train"],
                    eval_dataset=dataset["validation"],
                  )

trainer.train()

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/sdsldkfj...somefileid.. -O /some_parent_directory/some_child_directory

In [5]:
import gdown
url = 'https://drive.google.com/drive/folders/1aWSFb-ZR8TTIDdRlDBBCh-YvvCxmt6Bc'
output = '20150428_collected_images.tgz'
gdown.download(url, output, quiet=False)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/1aWSFb-ZR8TTIDdRlDBBCh-YvvCxmt6Bc
To: /content/20150428_collected_images.tgz
1.21MB [00:00, 65.0MB/s]


'20150428_collected_images.tgz'